The notebook will be an implementation of the finite difference of the (1D) wave equation on a rectangular domain

The 1D wave equation is a linear partial differential equation (PDE) of the form 
$$
    \frac{\partial^2 u}{\partial t^2} = D^2\frac{\partial^2 u}{\partial x^2}
$$
where $t$ is the time and $x$ is the direction.

To specify a unique solution $u(x,t)$ on the domain $[a,b] \times [0, \infty)$, one would need to provide an inital condition on both the solution, its time partial derivatives along and its boundary condition on the sides of the rectangle. That is, one would need
$$
    \begin{equation}
        \begin{cases}
            u(x,0) = f(x) \qquad \hspace{14pt} \text{ for } a \leq x \leq b \\
            \displaystyle \frac{\partial u}{\partial t}(x,0) = \frac{df}{dt}(x) \quad \hspace{8pt} \text{ for } a \leq x \leq b \\
            u(a,t) = l(t) \qquad \qquad \text{ for } t \geq 0 \\
            u(b,t) = r(t) \qquad \qquad \text{ for } t \geq 0
        \end{cases}
    \end{equation}
$$

To discretize it, we will first take a portion of time interval $[t_0, t] \subseteq [0,\infty)$ and work with the rectangle $[a,b] \times [t_0,t]$. Choose the step-size $h$ for the space and $k$ then to make the solution stable, these would need to sastisfy the <b> CFL-condition </b>
$$
    \alpha = \frac{Dk}{h} \leq 1
$$

In [7]:
import numpy as np

# Initial conditions
def f(x):
    return 0*x
def f_t(x):
    return 2*np.pi*np.sin(np.pi*x)
def l(t):
    return 0*t
def r(t):
    return 0*t
# Wave speed
D = 2 

In [59]:
# Finite difference of the wave equation (IN CONSTRUCTION)
def waveFD(f,f_t,l,r,D,x_0,x_end,t_0,t_end,space_steps,time_steps):
    h = (x_end - x_0)/space_steps
    k = (t_end - t_0)/time_steps
    alpha = (D*k)/h

    # Checking for stability
    if alpha < 0 or alpha > 1:
        print("Fail to meet the CFL-condition for stability")
        return
    
    m = int(space_steps - 1)
    n = int(time_steps)

    # Constructing the matrix needed to do finite-difference approximation 
    # (It's better to use sparse matrix for this, but I don't know how to do it for now)
    A = np.diag((2-2*(alpha**2))*np.ones(m)) + np.diag((alpha**2)*np.ones(m-1),k=1) + np.diag((alpha**2)*np.ones(m-1),k=-1)
    #return A #(Debugging)

    # Initialize resulting matrix
    W = np.zeros()
    

In [60]:
waveFD(f,f_t,l,r,D, x_0 = 0, x_end = 1, t_0 = 0, t_end = 1, space_steps=200, time_steps=1000)

array([[1.68, 0.16, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.16, 1.68, 0.16, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.16, 1.68, ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 1.68, 0.16, 0.  ],
       [0.  , 0.  , 0.  , ..., 0.16, 1.68, 0.16],
       [0.  , 0.  , 0.  , ..., 0.  , 0.16, 1.68]])